<a href="https://colab.research.google.com/github/JGranda11/Proyecto-ai4eng20241/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Preprocesamiento
En el siguiente cuaderno se quiere mostrar que realizacione sy consideraciones se tuvieron en cuenta para aplicar las operaciones de limpieza y preprocesado de datos.

## Cargar el Dataset

In [1]:
#Cargar el token de Kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jhonatangranda","key":"7dd2bc4d9b9fc9fcaeb5d991e5472b44"}'}

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [3]:
!kaggle competitions download udea-ai4eng-20241

 99% 21.0M/21.2M [00:00<00:00, 43.9MB/s]
100% 21.2M/21.2M [00:00<00:00, 39.9MB/s]


In [89]:
#Descomprimiendo los archivos
!unzip udea-ai4eng-20241

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [90]:
# Cargar dataset

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [91]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


#Consideraciones
Del proceso de exploración de los datos, se infiere que el dataset en su composición tiene:

* Columnas repetidas:
__FAMI_TIENEINTERNET__ y __FAMI_TIENEINTERNET.1__

Adeás necesito categorizar varias columnas que representan un orden en su estructura
* ESTU_VALORMATRICULAUNIVERSIDAD
* ESTU_HORASSEMANATRABAJA
* FAMI_ESTRATOVIVIENDA


In [92]:
#Se hace la asignación, para el cambio de nombre de la columna repetida
train = train.rename(columns={'FAMI_TIENEINTERNET.1': 'FAMI_TIENEINTERNET2'})

In [93]:
train.FAMI_TIENEINTERNET.value_counts()
#Columna original

FAMI_TIENEINTERNET
Si    592514
No     73357
Name: count, dtype: int64

In [94]:
train.FAMI_TIENEINTERNET2.value_counts()
#COLUMNA DUPLICADA

FAMI_TIENEINTERNET2
Si    592514
No     73357
Name: count, dtype: int64

In [96]:
validar = train['FAMI_TIENEINTERNET'].equals(train['FAMI_TIENEINTERNET2'])
print(validar)

True


In [97]:
train = train.drop('FAMI_TIENEINTERNET2', axis=1)

In [98]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Primaria completa,medio-bajo


In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_TIENELAVADORA              652727 non-null  object
 10  FAMI_TIENEAUTOMOVIL             648877 non-null  object
 11  ESTU_PRIVADO_LIBERTAD           692500 non-null  object
 12  ESTU_PAGOMATRICULAPROPIO      

Evaluación general de todos los datos nulos

In [99]:
k = train.isna().sum()
k[k!=0]


ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_EDUCACIONMADRE               23664
dtype: int64

## Manejo de la columna PERIODO
Esta columna presenta 9 distintas posibilidades de ocurrencia, se quiere corregir y agrupar de mejor manera por año.

In [173]:
train.PERIODO.value_counts()

PERIODO
20195    180873
20203    171838
20212    171412
20183    164818
20194      1472
20213      1178
20202       490
20184       254
20196       165
Name: count, dtype: int64

In [172]:
train['PERIODO'] = train['PERIODO'].astype('object')

In [168]:
train['PERIODO'] = train['PERIODO'].replace(r'2018.*', '2018', regex=True)
train['PERIODO'] = train['PERIODO'].replace(r'2019.*', '2019', regex=True)
train['PERIODO'] = train['PERIODO'].replace(r'2020.*', '2020', regex=True)
train['PERIODO'] = train['PERIODO'].replace(r'2021.*', '2021', regex=True)

## Manejo de la columna ESTU_PRGM_ACADEMICO

In [100]:
#La columna ESTU_PRGM_ACADEMICO es la que representa un mayor numero de variedad en el rango de sus posibilidades
train["ESTU_PRGM_ACADEMICO"].value_counts()

ESTU_PRGM_ACADEMICO
DERECHO                                                   53244
ADMINISTRACION DE EMPRESAS                                51902
CONTADURIA PUBLICA                                        39664
PSICOLOGIA                                                31932
INGENIERIA INDUSTRIAL                                     28481
                                                          ...  
LICENCIATURA EN ESPAÑOL Y FILOLOGÍA                           1
ADMINISTRACION EN NEGOCIOS INTERNACIONALES                    1
ADMINISTRACION DE COMERCIO EXTERIOR                           1
LICENCIATURA EN EDUCACI¿N F¿SICA RECREACI¿N Y DEPORTES        1
PROFESIONAL EN INVESTIGACIÓN CRIMINAL                         1
Name: count, Length: 948, dtype: int64

In [156]:
train['PERIODO'].value_counts()

PERIODO
20195    180873
20203    171838
20212    171412
20183    164818
20194      1472
20213      1178
20202       490
20184       254
20196       165
Name: count, dtype: int64

In [101]:
!pip install unidecode

Se hacen ciertas consideraciones para eliminar varios acentos, y caracteres es las posibles respuestas de la columna

In [102]:
from unidecode import unidecode
import re

train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].apply(lambda x: unidecode(x).strip()) # Eliminar tildes
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'\s+', ' ',x).strip()) # Eliminar dobles espacios
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].apply(lambda x: re.sub(r'^PROFESIONAL EN\s+', '', x).strip()) # "PROFESIONAL EN ..." redundante
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'INGENIER\?A', 'INGENIERIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'EDUCACI\?N\b', 'EDUCACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'RECREACI\?N\b', 'RECREACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'F\?SICA\b', 'FISICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ESC\?NICAS\b', 'ESCENICAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'MATEM\?TICAS\b', 'MATEMATICAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'FILOSOF\?A\b', 'FILOSOFIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'DISE\?O\b', 'DISEÑO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'DISE\??O\b', 'DISEÑO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACI\?N\b', 'COMUNICACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACI\??N\b', 'COMUNICACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GR\??FICA\b', 'GRAFICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'EL\?CTRICA\b', 'ELECTRICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'QU\?MICA\b', 'QUIMICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'FARMAC\?UTICA\b', 'FARMACEUTICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ADMINISTRACI\?N\b', 'ADMINISTRACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'P\?BLICA\b', 'PUBLICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'INGL\?S\b', 'INGLES', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'\?NFASIS\b', 'ENFASIS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GESTI\?N\b', 'GESTION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'M\?SICA\b', 'MUSICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ECONOM\?A\b', 'ECONOMIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GASTRONOM\?A\b', 'GASTRONOMIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'B\?SICA\b', 'BASICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GEOLOG\?A\b', 'GEOLOGIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'PEDAGOG\?A\b', 'PEDAGOGIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ART\?STICA\b', 'ARTISTICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'LOG\?STICA\b', 'LOGISTICA', regex=True)

In [19]:
train['ESTU_PRGM_ACADEMICO'].isnull().any()
#Se verifica que la columna no tenga valores nulos

False

In [103]:
# Reemplazar cualquier coincidencia de "{NOMBRE + AGG} ..." solo por "{NOMBRE} mas general"
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'INGENIERIA.*', 'INGENIERIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ADMINISTRACION.*', 'ADMINISTRACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'DISEÑO.*', 'DISEÑO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'DISENO.*', 'DISEÑO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'LICENCIATURA.*', 'LICENCIATURA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'CIENCIA.*', 'CIENCIAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'COMUNICACION.*', 'COMUNICACION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GESTION.*', 'GESTION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'FILOSOFIA.*', 'FILOSOFIA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'QUIMICA.*', 'QUIMICA', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'NEGOCIOS.*', 'NEGOCIOS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'MERCADEO.*', 'MERCADEO', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'MATEMATICAS.*', 'MATEMATICAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ARTE.*', 'ARTES', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'ESTUDIOS.*', 'ESTUDIOS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'FINANZAS.*', 'FINANZAS', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'CONSTRUCCION.*', 'CONSTRUCCION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'DIRECCION.*', 'DIRECCION', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'RELACIONES.*', 'RELACIONES', regex=True)
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'GEO.*', 'GEOGRAFIA', regex=True)


In [104]:
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'PROFESIONAL.*', 'PROFESIONAL', regex=True)

In [105]:
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'LENG.*', 'LENGUAS', regex=True)

In [106]:
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].replace(r'CONTADURIA.*', 'CONTADURIA', regex=True)

In [107]:
ccols = [i for i in train.columns if not i in train._get_numeric_data()]
print (ccols)

['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']


In [108]:
for c in ccols:
    print ("%10s"%c, np.unique(train[c].dropna()))

ESTU_PRGM_ACADEMICO ['3deg CICLO PROFESIONAL' 'ACTIVIDAD FISICA Y DEPORTE' 'ACUICULTURA'
 'ADMINISTRACION' 'ADMINSITRACION DE EMPRESAS'
 'ADMINSITRACION TURISTICA Y HOTELERA' 'AGROINDUSTRIA' 'AGRONEGOCIOS'
 'AGRONOMIA' 'AGRONOMIA DEL TROPICO HUMEDO' 'ANIMACION' 'ANTROPOLOGIA'
 'ARCHIVISTICA' 'ARQUEOLOGIA' 'ARQUITECTURA' 'ARTES' 'ASTRONOMIA'
 'BACTERIOLOGIA' 'BACTERIOLOGIA Y LABORATORIO CLINICO' 'BANCA Y FINANZAS'
 'BELLAS ARTES' 'BIBLIOTECOLOGIA' 'BIOINGENIERIA' 'BIOLOGIA'
 'BIOLOGIA AMBIENTAL' 'BIOLOGIA APLICADA' 'BIOLOGIA MARINA' 'BIOQUIMICA'
 'BIOTECNOLOGIA' 'CIENCIAS' 'CINE' 'CINE Y AUDIOVISUALES'
 'CINE Y COMUNICACION' 'CINE Y TELEVISION' 'COMERCIO EXTERIOR'
 'COMERCIO INTERNACIONAL' 'COMERCIO INTERNACIONAL Y FINANZAS'
 'COMERCIO Y FINANZAS' 'COMERCIO Y NEGOCIOS' 'COMPUTACION CIENTIFICA'
 'COMUNICACION' 'CONSERVACION Y RESTAURACION DE BIENES MUEBLES'
 'CONSTRUCCION' 'CONTADURIA' 'CREACION LITERARIA' 'CRIMINALISTICA'
 'CULINARIA Y GASTRONOMIA' 'CULTURA FISICA Y DEPORTE'
 'CULTURA F

In [109]:
academico=train['ESTU_PRGM_ACADEMICO'].unique()
len(academico)
#academico

182

## Manejo de la columna ESTU_VALORMATRICULAUNIVERSIDAD

In [31]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].unique()


array(['Entre 5.5 millones y menos de 7 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones', 'Más de 7 millones',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'No pagó matrícula', nan], dtype=object)

In [32]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

ESTU_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      204048
Entre 2.5 millones y menos de 4 millones    127430
Menos de 500 mil                             80263
Entre 500 mil y menos de 1 millón            78704
Entre 4 millones y menos de 5.5 millones     69736
Más de 7 millones                            68014
Entre 5.5 millones y menos de 7 millones     38490
No pagó matrícula                            19528
Name: count, dtype: int64

In [118]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()

6287

Tenemos una columna que presenta 8 posibles categorias, y que la categoria con mayor frecuencia, es un valor de matricula entre 1 y 2.5 M.

Se decide rellenar los datos faltantes, asumiendo que aquellos estudiantes que no tengan un valor asignado, es como si no realizaran un pago


In [119]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('No pagó matrícula')

In [120]:
#Realizamos el Encode para este columna
valores_matricula = ['No pagó matrícula','Menos de 500 mil','Entre 500 mil y menos de 1 millón',
                     'Entre 1 millón y menos de 2.5 millones',
                     'Entre 2.5 millones y menos de 4 millones',
                     'Entre 4 millones y menos de 5.5 millones',
                     'Entre 5.5 millones y menos de 7 millones','Más de 7 millones']

enc = OrdinalEncoder(categories = [valores_matricula])

train['ESTU_VALORMATRICULAUNIVERSIDAD'] = enc.fit_transform(train[['ESTU_VALORMATRICULAUNIVERSIDAD']])

In [121]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

ESTU_VALORMATRICULAUNIVERSIDAD
3.0    204048
4.0    127430
1.0     80263
2.0     78704
5.0     69736
7.0     68014
6.0     38490
0.0     25815
Name: count, dtype: int64

## Manejo de la columna ESTU_HORASSEMANATRABAJA
La columna presenta 5 categorias, y presenta 30857 datos nulos, por lo que no debo realizar alguna reparacion, paar este caso asumimos que si el campo esta vacio, es por que el estudiante no tiene unas horas de trabajo


In [38]:
train['ESTU_HORASSEMANATRABAJA'].unique()

array(['Menos de 10 horas', '0', 'Más de 30 horas', 'Entre 21 y 30 horas',
       'Entre 11 y 20 horas', nan], dtype=object)

In [39]:
train['ESTU_HORASSEMANATRABAJA'].value_counts()

ESTU_HORASSEMANATRABAJA
Más de 30 horas        249352
0                      116550
Entre 11 y 20 horas    115857
Entre 21 y 30 horas     92693
Menos de 10 horas       87191
Name: count, dtype: int64

In [125]:
train['ESTU_HORASSEMANATRABAJA'].isna().sum()

0

In [122]:
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].fillna('0')

In [123]:
horas_trabajo = ['0','Menos de 10 horas','Entre 11 y 20 horas',
                'Entre 21 y 30 horas',
                'Más de 30 horas']

enc = OrdinalEncoder(categories = [horas_trabajo])

train['ESTU_HORASSEMANATRABAJA'] = enc.fit_transform(train[['ESTU_HORASSEMANATRABAJA']])

In [124]:
train['ESTU_HORASSEMANATRABAJA'].value_counts()

ESTU_HORASSEMANATRABAJA
4.0    249352
0.0    147407
2.0    115857
3.0     92693
1.0     87191
Name: count, dtype: int64

## Manejo de la columna FAMI_ESTRATOVIVIENDA
Dicha columna presenta 7 categorias, y ademas de eso un total de 32137 datos nulos, la estrategia que se plantea para repaar dichos datos nulos se baso en ver cual era la categoria con mayor frecuencia

In [45]:
train['FAMI_ESTRATOVIVIENDA'].unique()

array(['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1',
       nan, 'Estrato 6', 'Sin Estrato'], dtype=object)

In [46]:
train['FAMI_ESTRATOVIVIENDA'].value_counts()

FAMI_ESTRATOVIVIENDA
Estrato 2      232671
Estrato 3      210685
Estrato 1      111991
Estrato 4       65514
Estrato 5       23608
Estrato 6       12605
Sin Estrato      3289
Name: count, dtype: int64

In [129]:
train['FAMI_ESTRATOVIVIENDA'].isna().sum()

0

In [127]:
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 2')

In [128]:
estratos = ['Sin Estrato','Estrato 1','Estrato 2',
            'Estrato 3', 'Estrato 4', 'Estrato 5',
            'Estrato 6']

enc = OrdinalEncoder(categories = [estratos])

train['FAMI_ESTRATOVIVIENDA'] = enc.fit_transform(train[['FAMI_ESTRATOVIVIENDA']])

In [50]:
train['FAMI_ESTRATOVIVIENDA'].value_counts()

FAMI_ESTRATOVIVIENDA
2.0    264808
3.0    210685
1.0    111991
4.0     65514
5.0     23608
6.0     12605
0.0      3289
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENEINTERNET
Para este caso, tengo solo dos posibles clases, por lo que debo codificar de manera binaria, ademas tengo 26629 datos nuelos, y se decide clasificar dichos datos que no aparecen, como un 'NO'

In [51]:
train['FAMI_TIENEINTERNET'].unique()

array(['Si', 'No', nan], dtype=object)

In [52]:
train['FAMI_TIENEINTERNET'].value_counts()

FAMI_TIENEINTERNET
Si    592514
No     73357
Name: count, dtype: int64

In [133]:
train['FAMI_TIENEINTERNET'].isna().sum()

0

In [130]:
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].fillna("No")

In [131]:
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].replace(r'Si', 1.)
train['FAMI_TIENEINTERNET'] = train['FAMI_TIENEINTERNET'].replace(r'No', 0.)

In [132]:
train['FAMI_TIENEINTERNET'].value_counts()

FAMI_TIENEINTERNET
1.0    592514
0.0     99986
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENELAVADORA
Esta columna tambien presenta una posibilidad de valores de 2 rangos, por lo que dicha encodificación va a ser binaria, ademas contienen 39773 registros nulos

In [58]:
train['FAMI_TIENELAVADORA'].unique()

array(['Si', 'No', nan], dtype=object)

In [60]:
train['FAMI_TIENELAVADORA'].value_counts()

FAMI_TIENELAVADORA
Si    563390
No     89337
Name: count, dtype: int64

In [136]:
train['FAMI_TIENELAVADORA'].isna().sum()

0

In [134]:
train['FAMI_TIENELAVADORA'] = train['FAMI_TIENELAVADORA'].fillna("No")

In [135]:
train['FAMI_TIENELAVADORA'] = train['FAMI_TIENELAVADORA'].replace(r'Si', 1.)
train['FAMI_TIENELAVADORA'] = train['FAMI_TIENELAVADORA'].replace(r'No', 0.)

In [64]:
train['FAMI_TIENELAVADORA'].value_counts()

FAMI_TIENELAVADORA
1.0    563390
0.0    129110
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENEAUTOMOVIL
La columna presenta una distinción entre valores de tipo binario, por lo que todos los datos pueden tomar solo dos valores, y presenta 43623 registros nulos

In [65]:
train['FAMI_TIENEAUTOMOVIL'].unique()

array(['Si', 'No', nan], dtype=object)

In [66]:
train['FAMI_TIENEAUTOMOVIL'].value_counts()

FAMI_TIENEAUTOMOVIL
No    412606
Si    236271
Name: count, dtype: int64

In [139]:
train['FAMI_TIENEAUTOMOVIL'].isna().sum()

0

In [137]:
train['FAMI_TIENEAUTOMOVIL'] = train['FAMI_TIENEAUTOMOVIL'].fillna("No")

In [138]:
train['FAMI_TIENEAUTOMOVIL'] = train['FAMI_TIENEAUTOMOVIL'].replace(r'Si', 1.)
train['FAMI_TIENEAUTOMOVIL'] = train['FAMI_TIENEAUTOMOVIL'].replace(r'No', 0.)

In [70]:
train['FAMI_TIENEAUTOMOVIL'].value_counts()

FAMI_TIENEAUTOMOVIL
0.0    456229
1.0    236271
Name: count, dtype: int64

## Manejo de la columna FAMI_TIENECOMPUTADOR
Comportmaiento binario.
Presenta registros nulos

In [72]:
train['FAMI_TIENECOMPUTADOR'].unique()

array(['Si', 'No', nan], dtype=object)

In [73]:
train['FAMI_TIENECOMPUTADOR'].value_counts()

FAMI_TIENECOMPUTADOR
Si    597670
No     56727
Name: count, dtype: int64

In [143]:
train['FAMI_TIENECOMPUTADOR'].isna().sum()

0

In [141]:
train['FAMI_TIENECOMPUTADOR'] = train['FAMI_TIENECOMPUTADOR'].fillna("No")

In [142]:
train['FAMI_TIENECOMPUTADOR'] = train['FAMI_TIENECOMPUTADOR'].replace(r'Si', 1.)
train['FAMI_TIENECOMPUTADOR'] = train['FAMI_TIENECOMPUTADOR'].replace(r'No', 0.)

In [77]:
train['FAMI_TIENECOMPUTADOR'].value_counts()

FAMI_TIENECOMPUTADOR
1.0    597670
0.0     94830
Name: count, dtype: int64

## Manejo de la columna ESTU_PAGOMATRICULAPROPIO
Esta columna presenta un rango binario en su conjunto de posibles valores, y un total de 6498 datos nulos.
Por el momento la decisión para hcaer la reparación, se elige de asignar 'NO' a los datos nulos

In [82]:
train['ESTU_PAGOMATRICULAPROPIO'].unique()

array(['No', 'Si', nan], dtype=object)

In [83]:
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

ESTU_PAGOMATRICULAPROPIO
No    382201
Si    303801
Name: count, dtype: int64

In [147]:
train['ESTU_PAGOMATRICULAPROPIO'].isna().sum()

0

In [144]:
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].fillna("No")

In [145]:
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].replace(r'Si', 1.)
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].replace(r'No', 0.)

In [146]:
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

ESTU_PAGOMATRICULAPROPIO
0.0    388699
1.0    303801
Name: count, dtype: int64

## Manejo de la columna ESTU_PRIVADO_LIBERTAD
Esta columna tiene un comportamiento binario, y no presenta datos nulos


In [111]:
train['ESTU_PRIVADO_LIBERTAD'].unique()

array(['N', 'S'], dtype=object)

In [112]:
train['ESTU_PRIVADO_LIBERTAD'].value_counts()

ESTU_PRIVADO_LIBERTAD
N    692466
S        34
Name: count, dtype: int64

In [149]:
train['ESTU_PRIVADO_LIBERTAD'].isna().sum()

0

In [148]:
train['ESTU_PRIVADO_LIBERTAD'] = train['ESTU_PRIVADO_LIBERTAD'].replace(r'S', 1.)
train['ESTU_PRIVADO_LIBERTAD'] = train['ESTU_PRIVADO_LIBERTAD'].replace(r'N', 0.)

In [150]:
train['ESTU_PRIVADO_LIBERTAD'].value_counts()

ESTU_PRIVADO_LIBERTAD
0.0    692466
1.0        34
Name: count, dtype: int64

In [151]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,6.0,1.0,3.0,1.0,Técnica o tecnológica incompleta,1.0,1.0,0.0,0.0,1.0,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,4.0,0.0,3.0,0.0,Técnica o tecnológica completa,1.0,0.0,0.0,0.0,1.0,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO,BOGOTÁ,4.0,4.0,3.0,1.0,Secundaria (Bachillerato) completa,1.0,0.0,0.0,0.0,0.0,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION,SANTANDER,5.0,0.0,4.0,1.0,No sabe,1.0,0.0,0.0,0.0,1.0,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,4.0,3.0,3.0,1.0,Primaria completa,1.0,1.0,0.0,0.0,1.0,Primaria completa,medio-bajo
